In [13]:
import requests
from bs4 import BeautifulSoup
import csv
import json

In [14]:
# 定義 PTT baseball 版的網址
url = 'https://www.ptt.cc/bbs/Baseball/index.html'

# 設定爬取的頁數
pages_to_crawl = 30

# 創建一个空列表来儲存文章資料
articles_list = []

#要搜尋的關鍵字
keywords = ["大谷翔平", "道奇", "大谷"]

In [15]:
# 使用 requests 取得網頁內容
response = requests.get(url)
response

<Response [200]>

In [16]:
# 創建一個Session來處理Cookies
session = requests.Session()

# 添加"Over18"的Cookie
over18_cookie = {'over18': '1'}
session.cookies.update(over18_cookie)

In [17]:
for i in range(pages_to_crawl):
    # 使用 requests 取得網頁內容
    response = session.get(url)
    
    # 檢查是否成功取得網頁內容
    if response.status_code != 200:
        print(f'無法取得頁面 {url}')
        break

    # 使用 BeautifulSoup 解析網頁內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 找出文章標題和連結
    articles = soup.find_all('div', class_='r-ent')
    for article in articles:
        title = article.find('div', class_='title').text.strip()
        
        # 檢查文章內容是否包含"刪除"或其他被刪除的關鍵字
        if "刪除" in title:
            continue  # 跳過已刪除的文章
         
        # 檢查文章内容是否包含"Honda"的關鍵字
        if any(keyword in title.lower() for keyword in keywords):
            article_link = article.find('a')['href']
            date = article.find('div', class_='date').text.strip()
            author = article.find('div', class_='author').text.strip()

            # 將文章資料存為字典
            article_info = {
                '標題': title,
                '日期': date,
                '作者': author,
            }
            articles_list.append(article_info)
            print(f'標題: {title}')
            print(f'日期: {date}')
            print(f'作者: {author}')
        
    # 找出前一頁的連結
    previous_page = soup.find('div', class_='btn-group-paging').find_all('a')[1]['href']
    url = 'https://www.ptt.cc' + previous_page

標題: [新聞] MLB／藍鳥簽大谷翔平失利！想改追Bellinger　未婚妻曬照舖
日期: 12/12
作者: saiulbb
標題: [討論] 回憶大谷這六年哪一場最值得看?
日期: 12/12
作者: kelvintube
標題: [討論] 道奇 = NBA湖人？
日期: 12/13
作者: renna038766
標題: [討論] 我對於大谷說的補強的看法
日期: 12/12
作者: Lemon487
標題: [新聞] MLB》大谷翔平最重要的人!日媒爆翻譯水原
日期: 12/12
作者: InoueNagi
標題: [討論] 大谷為什麼要7億
日期: 12/12
作者: MAYADD
標題: [討論] 道奇組日本隊有料嗎
日期: 12/12
作者: renna038766
標題: Re: [分享] 大谷翔平本來想要把薪資全部延遲支付
日期: 12/12
作者: Yuan610
標題: [討論] 山本如果去道奇 大谷以後會不會被基迷噓
日期: 12/12
作者: NYYLG
標題: Re: [討論] 大谷道奇這10年幾冠算成功?
日期: 12/12
作者: ngilisigoa
標題: [新聞] 大谷翔平經濟效益提升！日本贊助商賺到了
日期: 12/12
作者: hvariables
標題: Re: [分享] 大谷翔平本來想要把薪資全部延遲支付
日期: 12/12
作者: larryrock
標題: [討論] 大谷合約延遲付款
日期: 12/12
作者: gasiacruise
標題: Re: [分享] 大谷翔平本來想要把薪資全部延遲支付
日期: 12/12
作者: x6073123
標題: Re: [分享] 大谷翔平本來想要把薪資全部延遲支付
日期: 12/12
作者: autorad
標題: [討論] 道奇要拿二千多萬空間幹什麼大事呢？
日期: 12/12
作者: autorad
標題: Re: [分享] 大谷翔平本來想要把薪資全部延遲支付
日期: 12/12
作者: ligin
標題: [討論] 大谷會不會是想結婚了？
日期: 12/12
作者: aa1477888
標題: [討論] 大谷道奇這10年幾冠算成功?
日期: 12/12
作者: rappig
標題: Re: [新聞] 大谷7億約97％延遲支付可實拿更多？
日期: 12/12
作者: aas

In [18]:
# 將資料寫入CSV文件
with open('pttbaseball.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = ['標題', '日期', '作者']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for article_info in articles_list:
        writer.writerow(article_info)

In [19]:
# 將資料寫入JSON文件
with open('pttbaseball.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(articles_list, jsonfile, ensure_ascii=False, indent=4)